In this notebook, we will use Callbacks during training to save a model in .h5 extension so we can load and use at a later time.
Using callbacks is very handy when you have large dataset that takes hours to train the model and you have to save each model after an epoch. 

This is also helpful in cases of unexpected interruption, so we wont lose th eentire progress of training.

# Using callbacks during Training

### Data Preparation

In [1]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
# %matplotlib inline

from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

print(housing.feature_names)

from sklearn.model_selection import train_test_split
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, test_size = 0.2, random_state = 42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size = 0.3, random_state=42)


from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

np.random.seed(42)
tf.random.set_seed(42)

['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']


### Create a model

In [2]:
tf.keras.backend.clear_session()

input_ = tf.keras.layers.Input(shape=X_train.shape[1:])
hidden1 = tf.keras.layers.Dense(30, activation='relu')(input_)
hidden2 = tf.keras.layers.Dense(30, activation='relu')(hidden1)
concat = tf.keras.layers.concatenate([input_, hidden2])
output = tf.keras.layers.Dense(1)(concat)
model_functional = tf.keras.models.Model(inputs=[input_], outputs=[output])

model_functional.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 30)           270         input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 30)           930         dense[0][0]                      
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 38)           0           input_1[0][0]                    
                                                                 dense_1[0][0]         

In [3]:
model_functional.compile(loss='mse',
                         optimizer=tf.keras.optimizers.SGD(lr=1e-3),
                        metrics=['mae'])

In [4]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('Model_Func_{epoch:02d}.h5')

In [5]:
model_history = model_functional.fit(X_train, y_train, epochs=10,
                                     validation_data=(X_valid, y_valid),
                                    callbacks=[checkpoint_cb])

Epoch 1/10
362/362 [==============================] - 0s 871us/step - loss: 1.2605 - mae: 0.7982 - val_loss: 0.7210 - val_mae: 0.6104
Epoch 2/10
362/362 [==============================] - 0s 599us/step - loss: 0.6456 - mae: 0.5819 - val_loss: 0.6469 - val_mae: 0.5727
Epoch 3/10
362/362 [==============================] - 0s 596us/step - loss: 0.6041 - mae: 0.5605 - val_loss: 0.6127 - val_mae: 0.5564
Epoch 4/10
362/362 [==============================] - 0s 593us/step - loss: 0.5759 - mae: 0.5465 - val_loss: 0.5814 - val_mae: 0.5418
Epoch 5/10
362/362 [==============================] - 0s 596us/step - loss: 0.5538 - mae: 0.5349 - val_loss: 0.5646 - val_mae: 0.5332
Epoch 6/10
362/362 [==============================] - 0s 593us/step - loss: 0.5369 - mae: 0.5259 - val_loss: 0.5500 - val_mae: 0.5272
Epoch 7/10
362/362 [==============================] - 0s 593us/step - loss: 0.5206 - mae: 0.5189 - val_loss: 0.5288 - val_mae: 0.5141
Epoch 8/10
362/362 [==============================] - 0s 597us

In [6]:
del model_functional
tf.keras.backend.clear_session()

In [7]:
model = tf.keras.models.load_model('Model_Func_10.h5')
mse_test = model.evaluate(X_test, y_test)

129/129 [==============================] - 0s 356us/step - loss: 0.5152 - mean_absolute_error: 0.5055


# Create callback for only best model

In [8]:
tf.keras.backend.clear_session()

input_ = tf.keras.layers.Input(shape=X_train.shape[1:])
hidden1 = tf.keras.layers.Dense(30, activation='relu')(input_)
hidden2 = tf.keras.layers.Dense(30, activation='relu')(hidden1)
concat = tf.keras.layers.concatenate([input_, hidden2])
output = tf.keras.layers.Dense(1)(concat)
model_functional = tf.keras.models.Model(inputs=[input_], outputs=[output])

model_functional.compile(loss='mse',
                         optimizer=tf.keras.optimizers.SGD(lr=1e-3),
                        metrics=['mae'])

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('Model_Func_Best.h5', save_best_only=True)

In [9]:
model_history = model_functional.fit(X_train, y_train, epochs=10,
                                     validation_data=(X_valid, y_valid),
                                    callbacks=[checkpoint_cb])

Epoch 1/10
362/362 [==============================] - 0s 798us/step - loss: 1.8109 - mae: 0.9158 - val_loss: 0.8412 - val_mae: 0.6639
Epoch 2/10
362/362 [==============================] - 0s 598us/step - loss: 0.7450 - mae: 0.6299 - val_loss: 0.7116 - val_mae: 0.6122
Epoch 3/10
362/362 [==============================] - 0s 597us/step - loss: 0.6750 - mae: 0.5968 - val_loss: 0.6596 - val_mae: 0.5876
Epoch 4/10
362/362 [==============================] - 0s 600us/step - loss: 0.6270 - mae: 0.5729 - val_loss: 0.6166 - val_mae: 0.5635
Epoch 5/10
362/362 [==============================] - 0s 600us/step - loss: 0.5896 - mae: 0.5543 - val_loss: 0.5901 - val_mae: 0.5489
Epoch 6/10
362/362 [==============================] - 0s 595us/step - loss: 0.5610 - mae: 0.5389 - val_loss: 0.5595 - val_mae: 0.5358
Epoch 7/10
362/362 [==============================] - 0s 601us/step - loss: 0.5367 - mae: 0.5275 - val_loss: 0.5332 - val_mae: 0.5188
Epoch 8/10
362/362 [==============================] - 0s 619us

# Get and use the best model

In [11]:
tf.keras.backend.clear_session()

In [12]:
del model_functional
model = tf.keras.models.load_model('Model_Func_Best.h5')
mse_test = model.evaluate(X_test, y_test)

129/129 [==============================] - 0s 360us/step - loss: 0.5126 - mean_absolute_error: 0.5044


# Create best model using early stopping

In [13]:
del model
tf.keras.backend.clear_session()

input_ = tf.keras.layers.Input(shape=X_train.shape[1:])
hidden1 = tf.keras.layers.Dense(30, activation='relu')(input_)
hidden2 = tf.keras.layers.Dense(30, activation='relu')(hidden1)
concat = tf.keras.layers.concatenate([input_, hidden2])
output = tf.keras.layers.Dense(1)(concat)
model_functional = tf.keras.models.Model(inputs=[input_], outputs=[output])

model_functional.compile(loss='mse',
                         optimizer=tf.keras.optimizers.SGD(lr=1e-3),
                        metrics=['mae'])

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('Early_Stop_Model.h5', save_best_only=True)

In [14]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, 
                                                     restore_best_weights=True)

In [15]:
model_history = model_functional.fit(X_train, y_train, epochs=500,
                                     validation_data=(X_valid, y_valid),
                                    callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/500
362/362 [==============================] - 0s 821us/step - loss: 1.9151 - mae: 0.9780 - val_loss: 0.8447 - val_mae: 0.6347
Epoch 2/500
362/362 [==============================] - 0s 617us/step - loss: 0.6995 - mae: 0.6068 - val_loss: 0.6777 - val_mae: 0.5927
Epoch 3/500
362/362 [==============================] - 0s 610us/step - loss: 0.6423 - mae: 0.5840 - val_loss: 0.6339 - val_mae: 0.5738
Epoch 4/500
362/362 [==============================] - 0s 622us/step - loss: 0.6089 - mae: 0.5675 - val_loss: 0.6034 - val_mae: 0.5582
Epoch 5/500
362/362 [==============================] - 0s 621us/step - loss: 0.5825 - mae: 0.5544 - val_loss: 0.5845 - val_mae: 0.5494
Epoch 6/500
362/362 [==============================] - 0s 623us/step - loss: 0.5621 - mae: 0.5435 - val_loss: 0.5598 - val_mae: 0.5376
Epoch 7/500
362/362 [==============================] - 0s 628us/step - loss: 0.5430 - mae: 0.5348 - val_loss: 0.5419 - val_mae: 0.5255
Epoch 8/500
362/362 [==============================] - 

362/362 [==============================] - 0s 594us/step - loss: 0.3688 - mae: 0.4322 - val_loss: 0.3738 - val_mae: 0.4346
Epoch 62/500
362/362 [==============================] - 0s 612us/step - loss: 0.3679 - mae: 0.4319 - val_loss: 0.3709 - val_mae: 0.4339
Epoch 63/500
362/362 [==============================] - 0s 600us/step - loss: 0.3668 - mae: 0.4313 - val_loss: 0.3749 - val_mae: 0.4357
Epoch 64/500
362/362 [==============================] - 0s 594us/step - loss: 0.3666 - mae: 0.4309 - val_loss: 0.3710 - val_mae: 0.4326
Epoch 65/500
362/362 [==============================] - 0s 616us/step - loss: 0.3655 - mae: 0.4299 - val_loss: 0.3684 - val_mae: 0.4339
Epoch 66/500
362/362 [==============================] - 0s 602us/step - loss: 0.3646 - mae: 0.4297 - val_loss: 0.3694 - val_mae: 0.4297
Epoch 67/500
362/362 [==============================] - 0s 613us/step - loss: 0.3639 - mae: 0.4289 - val_loss: 0.3659 - val_mae: 0.4296
Epoch 68/500
362/362 [==============================] - 0s 62

362/362 [==============================] - 0s 644us/step - loss: 0.3360 - mae: 0.4090 - val_loss: 0.3445 - val_mae: 0.4138
Epoch 122/500
362/362 [==============================] - 0s 604us/step - loss: 0.3356 - mae: 0.4085 - val_loss: 0.3443 - val_mae: 0.4138
Epoch 123/500
362/362 [==============================] - 0s 600us/step - loss: 0.3354 - mae: 0.4086 - val_loss: 0.3458 - val_mae: 0.4140
Epoch 124/500
362/362 [==============================] - 0s 637us/step - loss: 0.3349 - mae: 0.4080 - val_loss: 0.3409 - val_mae: 0.4121
Epoch 125/500
362/362 [==============================] - 0s 630us/step - loss: 0.3343 - mae: 0.4081 - val_loss: 0.3433 - val_mae: 0.4113
Epoch 126/500
362/362 [==============================] - 0s 637us/step - loss: 0.3340 - mae: 0.4071 - val_loss: 0.3402 - val_mae: 0.4103
Epoch 127/500
362/362 [==============================] - 0s 600us/step - loss: 0.3329 - mae: 0.4069 - val_loss: 0.3460 - val_mae: 0.4155
Epoch 128/500
362/362 [==============================] 

Epoch 181/500
362/362 [==============================] - 0s 637us/step - loss: 0.3173 - mae: 0.3943 - val_loss: 0.3245 - val_mae: 0.3990
Epoch 182/500
362/362 [==============================] - 0s 631us/step - loss: 0.3174 - mae: 0.3939 - val_loss: 0.3243 - val_mae: 0.4014
Epoch 183/500
362/362 [==============================] - 0s 615us/step - loss: 0.3165 - mae: 0.3941 - val_loss: 0.3275 - val_mae: 0.4033
Epoch 184/500
362/362 [==============================] - 0s 607us/step - loss: 0.3169 - mae: 0.3943 - val_loss: 0.3271 - val_mae: 0.4020
Epoch 185/500
362/362 [==============================] - 0s 604us/step - loss: 0.3165 - mae: 0.3941 - val_loss: 0.3254 - val_mae: 0.3982
Epoch 186/500
362/362 [==============================] - 0s 607us/step - loss: 0.3162 - mae: 0.3940 - val_loss: 0.3259 - val_mae: 0.3993
Epoch 187/500
362/362 [==============================] - 0s 604us/step - loss: 0.3164 - mae: 0.3934 - val_loss: 0.3278 - val_mae: 0.4043
Epoch 188/500
362/362 [==================

In [16]:
del model_functional
model = tf.keras.models.load_model('Early_Stop_Model.h5')
mse_test = model.evaluate(X_test, y_test)

129/129 [==============================] - 0s 358us/step - loss: 0.3279 - mean_absolute_error: 0.3987
